In [42]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.parse
import time

In [14]:

response = requests.get(page_url)

with open("home_page.html", "w", encoding="utf-8") as file:
    file.write(response.text)

In [15]:
with open("home_page.html", "r", encoding="utf-8") as file:
    html = file.read()

links = [a["href"] for a in soup.find_all("a", href=True)]
print("Extracted links:", links)

Extracted links: ['/en/3688006/index.html', 'javascript:void(0);', '/english/130437/index.html', 'http://www.pbc.gov.cn/', '/en/3688066/3688080/index.html', '/en/3688066/3688080/index.html', '/en/3688066/3688083/index.html', '/en/3688066/3688086/index.html', '/en/3688066/3688089/index.html', '/en/3688066/3688095/index.html', '/en/3688110/3688172/index.html', '/en/3688110/3688172/index.html', '/en/3688110/3688175/index.html', '/en/3688110/3688178/index.html', '/en/3688110/3688181/index.html', '/en/3688247/3688978/index.html', '/en/3688110/3688259/3689026/index.html', '/en/3688110/3688259/3689026/index.html', '/en/3688110/3688259/3689026/3706088/index.html', '/en/3688110/3688259/3689026/3706088/5188168/index.html', '/en/3688110/3688259/3689026/3706088/4756448/index.html', '/en/3688110/3688259/3689026/3706088/4601725/index.html', '/en/3688110/3688259/3689026/3706088/4437187/index.html', '/en/3688110/3688259/3689026/3706088/3982929/index.html', '/en/3688110/3688259/3689026/3706088/3830453/

In [31]:
home_page="http://www.pbc.gov.cn/en/3688247/3688975/index.html"
base_url="http://www.pbc.gov.cn/"


In [47]:
data_links=collect_links(home_page,base_url)
data_links_testing=data_links[10:20]
print(data_links_testing)

[]


In [ ]:
data_links_testing=data_links[10:20]

In [43]:
def scrape_pages(links):
    for url in links:
        print(f"Scraping: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Example: Extract titles or file download links
        title = soup.find("h1").text if soup.find("h1") else "No Title"
        downloads = [a["href"] for a in soup.find_all("a", href=True) if a["href"].endswith((".xlsx", ".pdf","htm"))]

        print(f"Title: {title}")
        print(f"Downloads: {downloads}")
        
        # Introduce a delay to avoid being blocked
        time.sleep(2)


In [ ]:
download_path="download"
def download_file(file_url):
    """Downloads a file and saves it in the designated folder."""
    filename = os.path.join(download_path, file_url.split("/")[-1])
    
    try:
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Raise error for bad responses (404, 500, etc.)
        
        with open(filename, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

        print(f"Downloaded: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {file_url}: {e}")

def process_download_links(download_links):
    """
    Processes download links, prioritizing Excel (.xls, .xlsx) over PDF.
    Assumes links for a single item are grouped together.
    """
    file_groups = {}  # Store files by item name (without extension)

    for link in download_links:
        file_name = link.split("/")[-1]
        base_name, ext = os.path.splitext(file_name)

        if base_name not in file_groups:
            file_groups[base_name] = {"xls": None, "pdf": None}
        
        if ext in [".xls", ".xlsx"]:
            file_groups[base_name]["xls"] = link
        elif ext == ".pdf":
            file_groups[base_name]["pdf"] = link

    # Download files, prioritizing Excel
    for base_name, files in file_groups.items():
        if files["xls"]:
            download_file(files["xls"])
        elif files["pdf"]:
            download_file(files["pdf"])

In [62]:
home_page="http://www.pbc.gov.cn/en/3688247/3688975/index.html"
base_url="http://www.pbc.gov.cn/"
def collect_links(home_page,base_url):
    response=requests.get(home_page)
    soup=bs(response.text, 'html.parser')
    page_links=[]
    for link in soup.find_all('a', href=True):
        href=link['href']
        if href.endswith('index.html') and "en/3688247/" in href:
            full_url = urllib.parse.urljoin(base_url, href) 
            page_links.append(full_url)
    return page_links
    

                
def organize_links(home_page, page_html):
    links = {}  # Dictionary to store links grouped by year
    years = [str(year) for year in range(2006, 2025)]  # Convert to strings for comparison

    # Parse the page
    soup = BeautifulSoup(page_html, "html.parser")

    # Extract meta description to determine the year
    meta_tag = soup.find("meta", attrs={"name": "description"})
    
    if meta_tag and "content" in meta_tag.attrs:
        page_title_year = meta_tag["content"].strip()  # Ensure no extra spaces
        
        if page_title_year in years:  # Check if it's a valid year
            print("Page Title Year:", page_title_year)
            
            # Collect all valid links
            year_links = []
            for link in soup.find_all('a', href=True):
                collected_link = collect_links(home_page, link['href'])  # Assuming collect_links() is defined
                year_links.append(collected_link)
            
            # Store links in dictionary under the correct year
            links[page_title_year] = year_links

    return links                
    
def get_download_links(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    meta_tag = soup.find("meta", attrs={"name": "description"})
    if meta_tag and "content" in meta_tag.attrs:
        extracted_value = meta_tag["content"]
        print("page_title_year:", extracted_value)
        
        
    if meta_tag and "content" in meta_tag.attrs:
        extracted_value = meta_tag["content"]
        print("page_title_year:", extracted_value)

    download_links = {}

    for link in soup.find_all("a", href=True):
        href = link["href"]
        if href.endswith((".xls", ".xlsx", ".pdf")):
            full_url = urllib.parse.urljoin(base_url, href)  # Convert relative URL to absolute
            download_links.append(full_url)

    return download_links 


In [49]:
data_links=collect_links(home_page,base_url)
data_links_testing=data_links[10:20]
print(data_links_testing)

<a class="npbcLogo" href="/en/3688006/index.html">
<img src="/en/template/common/newheader/20210205.png"/>
<span></span>
</a>
<a href="javascript:void(0);" id="phoneNav"></a>
<a href="/english/130437/index.html" target="_blank">Old Version</a>
<a href="http://www.pbc.gov.cn/" target="_blank">ä¸­æ</a>
<a class="headNavOneLiA" href="/en/3688066/3688080/index.html">About PBC</a>
<a class="headNavTwoLiA" href="/en/3688066/3688080/index.html">Purposes and Functions</a>
<a class="headNavTwoLiA" href="/en/3688066/3688083/index.html">Management Team</a>
<a class="headNavTwoLiA" href="/en/3688066/3688086/index.html">Former Governors</a>
<a class="headNavTwoLiA" href="/en/3688066/3688089/index.html">History</a>
<a class="headNavTwoLiA" href="/en/3688066/3688095/index.html">Organizational Structure</a>
<a class="headNavOneLiA" href="/en/3688110/3688172/index.html">News &amp; Events</a>
<a class="headNavTwoLiA" href="/en/3688110/3688172/index.html">Press Releases</a>
<a class="headNavTwoLiA" hre

In [63]:
links_organized=organize_links(data_links_testing)

TypeError: organize_links() missing 1 required positional argument: 'page_html'

In [53]:
all_links=[]
for url in data_links_testing:
    links=get_download_links(url)
    all_links.append(links)


In [54]:
print(all_links)

[[], ['http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/02/2024021917193120091.xlsx', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/02/2024021917193146513.pdf', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/02/2024021917193153911.xlsx', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/02/2024021917193173502.pdf'], ['http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/01/2024010809394722013.xlsx', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/01/2024010809394728213.pdf', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/01/2024011714335841046.xlsx', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/01/2024011714335813754.pdf', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2024/01/2024011714335851539.xlsx', 'http://www.pbc.gov.cn/eportal/fileDir/diaochatongjisi/resource/cms/2

In [ ]:
ef process_download_links(download_links):

    file_groups = {}  # Store files by item name (without extension)
    for link in download_links:

        
    for link in download_links:
        file_name = link.split("/")[-1]
        base_name, ext = os.path.splitext(file_name)

        if base_name not in file_groups:
            file_groups[base_name] = {"xls": None, "pdf": None}
        
        if ext in [".xls", ".xlsx"]:
            file_groups[base_name]["xls"] = link
        elif ext == ".pdf":
            file_groups[base_name]["pdf"] = link

    # Download files, prioritizing Excel
    for base_name, files in file_groups.items():
        if files["xls"]:
            download_file(files["xls"])
        elif files["pdf"]:
            download_file(files["pdf"])   

In [60]:
page_url="http://www.pbc.gov.cn/en/3688247/3688975/3984235/4503866/index.html"
response = requests.get(page_url)
soup=bs(response.text, 'html.parser')
print(soup)
meta_tag = soup.find("meta", attrs={"name": "description"})
if meta_tag and "content" in meta_tag.attrs:
    extracted_value = meta_tag["content"]
    print("Extracted Value:", extracted_value)  #

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><title>Aggregate Financing to the Real Economy</title>
<meta content="2023-12-14 16:15:33" name="é¡µé¢çææ¶é´"/>
<meta content="2023-12-04 15:34:59" name="ç¼å­æ¸çæ¶é´"/>
<meta content="7.9.5" name="easysiteçæ¬"/>
<meta content="" name="keywords"/>
<meta content="åå§åé¢éï¼Aggregate Financing to the Real" name="description"/>
<link href="/en/uiFramework/huilan-jquery-ui/css/huilan-jquery-ui.css" rel="stylesheet" type="text/css"/>
<script src="/en/uiFramework/huilan-jquery-ui/js/huilan-jquery-ui.js?self=true&amp;skin=default" type="text/javascript"></script>
<meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/><link href="	
/en/uiFramework/commonResource/image/logo.ico" rel="shortcut icon" type="image

In [ ]:
def organize_links(home_page,page_links):
    links={}
    years = list(range(2006, 2025))
    for link in soup.find_all('a', href=True):
        if meta_tag and "content" in meta_tag.attrs:
            if meta_tag['content'] in years:
                page_title_year = meta_tag["content"]
                print("page_title_year:", page_title_year)
                year_links=collect_links(home_page,link)
                links.append(page_title_year,year_links)